In [13]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import coco
import utils
import model as modellib
from model import log
import coco
import visualize
import matplotlib.pyplot as plt
import json
from pycocotools import mask as maskUtils

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

# Create Model & Load Weight

In [14]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
# config.display()

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=MODEL_DIR)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

# Prepare COCO categories

In [15]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

# Prepare VIDEOtag, COCO category 맵핑 데이타

In [16]:
# Load VIDEOtag, COCO category 맵핑 데이타
map_data = open("assets/json/category.json").read()
tmp_videotag_coco_cate_map = json.loads(map_data)["categories"]

# dictionary로 생성
videotag_coco_cate_map = {}
for cate_data in tmp_videotag_coco_cate_map:
    videotag_coco_cate_map[cate_data['id']] = cate_data

# Prepare VIDEOtag Dataset

### 0101~0102 dataset
https://s3.ap-northeast-2.amazonaws.com/f-machine-learning-dataset/datasets/fingerplus/0101_0102.zip

In [45]:
json_data = open("dataset/videotag/0101_0102/instances.json").read()
data = json.loads(json_data)

videotag_anno = data['annotations']
videotag_imgs = data['images']
videotag_cate = data['categories']


videotag_img_anno_ids = {}
videotag_anno_dic = {}
videotag_cate_dic = {}

for anno in videotag_anno:
    videotag_anno_dic[anno['id']] = anno
    
for cate in videotag_cate:
    videotag_cate_dic[cate['id']] = cate
        
for image in videotag_imgs:
    anno_ids = []
    image_id = image['id']
    
    for anno in videotag_anno:
        anno_img_id = anno['image_id']
        anno_id = anno['id']
        if anno_img_id == image_id:
            anno_ids.append(anno_id)
    # if not len(annos) == 0:
    videotag_img_anno_ids[image_id] = anno_ids

In [52]:
print(videotag_imgs[1]['id'])
print(videotag_img_anno_ids[videotag_imgs[1]['id']])

199606
['297282', '297283']


# Detect & Extract Region

annotations : [{
  area,         // Number - 영역 넓이(w*h)
  bbox,         // [x,y,width,height], (Array(4))
  category_id,  // int
  id,           // int
  image_id,     // int
  iscrowd,      // 0 or 1 (get anns for given crowd label (False or True))
  segmentation  // Array - mask 데이터 (RLE or [polygon])
}

In [47]:
def getAnnosByIds(anno_ids):
    result = []
    for anno_id in anno_ids:  
        result.append(videotag_anno_dic[anno_id])
        
    return result

def saveBbox(anno_id, bbox):
    for anno in videotag_anno:
        if anno['id'] == anno_id:
            anno['bbox'] = bbox
            print('saved bbox', anno)
            break

extract_annotations = []


###################################################################
# 모든 이미지에 대하여 predict를 실시한다.
###################################################################
vt_img_total = len(videotag_imgs)
for vt_img_current, vt_img in enumerate(videotag_imgs): 
    
    if vt_img_current > 2:
        break;
    
    print(str(vt_img_current)+'/'+str(vt_img_total))
    img_id = vt_img['id']
    
    
    ###################################################################
    # videotag image에 등록되어 있느 annotation이 있는지 확인 후, 로드
    ###################################################################
    vt_annos = getAnnosByIds(videotag_img_anno_ids[img_id])
    
    if len(vt_annos) == 0:
        continue
    
    
    ###################################################################
    # videotag image이 로드
    ###################################################################
    img_path = os.path.join(ROOT_DIR, 'dataset/videotag/0101_0102/images', vt_img['file_name']) 
    image = skimage.io.imread(img_path)
    # If grayscale. Convert to RGB for consistency.
    if image.ndim != 3:
        image = skimage.color.gray2rgb(image)
        
    height, width = image.shape[:2]    
        
    
    ###################################################################
    # Run detection
    ###################################################################
    results = model.detect([image], verbose=0)

    # Result
    r = results[0]
    
    rois = r['rois'] # y1, x1, y2, x2
    scores = r['scores']
    masks = r['masks'] # (height, width, 검출된 데이타 갯수)
    class_ids = r['class_ids']
    
    
    # 확인용
    confirm_coco_classes = [];
    confirm_vt_classes = [];
    confirm_vt_anno_points = []
    for vt_anno in vt_annos:
        point_x = int(width*vt_anno['x_pos'])
        point_y = int(height*vt_anno['y_pos'])
        confirm_vt_anno_points.append([point_y-5, point_x-5, point_y+5, point_x+5])
        
        vt_cate_id = vt_anno['category_id']
        vt_cate_data = videotag_cate_dic[vt_cate_id]
        confirm_vt_classes.append(vt_cate_data['name'])
                
    is_found = 0
    
    ###################################################################
    # 추출된 Annotation을 VIDEOtag Annotation과 비교
    ###################################################################
    for index, score in enumerate(scores):
#         if score < 0.9:
#             continue
        
        image_id = image_id
        category_id = class_ids[index]
        category_name = class_names[category_id]
        roi = rois[index]
        bbox = [roi[1], roi[0], roi[3] - roi[1], roi[2] - roi[0]]
        mask = masks[:, :, index]
        '''
        segmentation = maskUtils.encode(np.asfortranarray(mask))
        rle = maskUtils.decode(segmentation)
#         m = maskUtils.decode(rle)
        print('segmentation', rle)
        area = maskUtils.area(segmentation)
        '''
        annotation = {}
        annotation['bbox'] = bbox
        annotation['category_id'] = category_id
        annotation['id'] = ''
        annotation['image_id'] = image_id
        annotation['iscrowd'] = 0
        '''
        annotation['area'] = area
        annotation['segmentation'] = segmentation
        '''
        # 확인용
        confirm_coco_classes.append(category_name)
        
        ###################################################################
        # coco model에서 검출된 annotation들이 videotag annotation(정답)들과 비교하여
        # 일치할 경우가 있을 때, 해당 videotag annotation에 bbox정보를 추가한다.
        ###################################################################
        for vt_anno in vt_annos:
            vt_anno_id = vt_anno['id']
            vt_cate_id = vt_anno['category_id']
            vt_cate_data = videotag_cate_dic[vt_cate_id]
            vt_cate_name = vt_cate_data['name']
            
            point_x = int(width*vt_anno['x_pos'])
            point_y = int(height*vt_anno['y_pos'])
            
            coco_cate_ids = videotag_coco_cate_map[vt_cate_id]['coco_ids']
            if len(coco_cate_ids) == 0:
                continue
                
            # NOTE: 포인트가 마스크 영역에 속하는 확인
            if mask[point_y][point_x] == 0:
                continue
            
#             vt_anno['bbox'] = bbox
            saveBbox(vt_anno_id, bbox)
            extract_annotations.append(annotation)
            
            for coco_cate_id in coco_cate_ids:
                if coco_cate_id == category_id:
                    print('찾았다', 'videotag : '+vt_cate_name, 'coco : '+category_name)
#                     vt_anno['bbox'] = [1,1,1,1]
                    is_found = 1
                    break
                    
    print('결과값 비교 coco', confirm_coco_classes)
    print('결과값 비교 videotag', confirm_vt_classes)
    print('----------------------------------------------------------------------')
    
    if is_found:
        visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'], confirm_videotag_anno_points)
        
import json
with open('dataset/videotag/0101_0102/videotag_anno.json', 'w') as outfile:
    json.dump(videotag_anno, outfile)

with open('dataset/videotag/0101_0102/extract_annotations.json', 'w') as outfile:
    json.dump(extract_annotations, outfile)

0/1081
1/1081
결과값 비교 coco ['person']
결과값 비교 videotag ['모자', '재킷']
----------------------------------------------------------------------
2/1081
saved bbox {'id': '297292', 'x_pos': 0.6097560975609756, 'y_pos': 0.7074303405572755, 'image_id': '199607', 'category_id': 'B09', 'bbox': [268, 148, 168, 200]}
결과값 비교 coco ['person', 'person', 'person', 'chair', 'handbag', 'cell phone']
결과값 비교 videotag ['시계', '선글라스']
----------------------------------------------------------------------


TypeError: Object of type 'int32' is not JSON serializable